In [ ]:
from airflow import DAG
from datetime import datetime,timedelta
from airflow.operators.python import PythonOperator

## Projects
import pandas as pd
import numpy as np
import joblib
import os
import time
import datetime
from datetime import timedelta,date,datetime
import configparser as cp
from sqlalchemy import create_engine
import mysql.connector

# 
import math
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Environment Set-up

In [ ]:
## Airflow ##
linear_analysis_arg={'owner':'airflow',
                     'depends_on_past':False,
                     'start_date':datetime.datetime(2022,3,1),
                     'retries':1,
                     'retry_delay':timedelta(minutes=10)
                    }

## Database ##
config=cp.ConfigParser()
config.read('/home/ubuntu/certi/db_login.txt')
db_config=config['ivan_db']

engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))

# Data Loading & Cleaning
* Save the final result as pkl

In [ ]:
def stock_data_cleaning():
    ## Data Loading ##
    # 01 Stock mapping
    stock_mapping=pd.read_sql("""SELECT * 
                                 FROM STOCK_PRED.NYSE_NASDAQ_TICKERS
                              """,con=engine)
    print(stock_mapping.shape)
    print(stock_mapping.Symbol.nunique())

    # 02 Stock data
    df=pd.read_sql("""SELECT * 
                      FROM STOCK_PRED.ALL_STOCK_HIST
                      WHERE DATE>=STR_TO_DATE('2020-01-01','%Y-%m-%d')
                             """,
                   con=engine)
    print(df.shape)
    print(df['Date'].max(),df['Date'].min())
    
    